In [7]:
import pandas as pd
import anndata as ad
import scanpy as sc
import numpy as np
import scrublet
import matplotlib.pyplot as plt
import os
import gzip

#**count矩阵读取及样本拆分**

In [8]:
# 读取count矩阵文件
count_matrix = pd.read_csv('/data/wenjiayue/Data/smart-seq/GSE198291/GSE198291_allcounts2.csv.gz', sep=',', index_col=0)
# https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE198291&format=file&file=GSE198291%5Fallcounts2%2Ecsv%2Egz
count_matrix

,PT1_MTT1,PT1_MTT2,PT1_MTT3,PT1_MTT4,PT1_MTT5,PT1_MTT6,PT1_MTT7,PT1_MTT8,PT1_MTT9,PT1_MTT10,...,PT9_MTT39,PT9_MTT40,PT9_MTT41,PT9_MTT42,PT9_MTT43,PT9_MTT44,PT9_MTT45,PT9_MTT46,PT9_MTT47,PT9_MTT48
WASH7P,0,1,0,0,0,1,0,2,4,6,...,6,3,0,4,1,5,0,0,0,0
FAM138A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FAM138F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LOC100132287,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDY1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CSPG4P1Y,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CSPG4P2Y,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GOLGA2P2Y,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# 观察列名，以第一列为例，认为 PT1_MTT1 对应一个细胞（cell），其来源的组织标本（sample）是 PT1_MTT，组织类型为 MTT (metastatic tumor tissue)
column_names = [name[:7] for name in count_matrix.columns.tolist()]

# 列出所有组织标本（sample）
unique_column_names = list(set(column_names))
unique_column_names


['PT7_MTP',
 'PT9_MTT',
 'PT3_PTT',
 'PT4_PTT',
 'PT9_MTP',
 'PT4_MTP',
 'PT1_PTT',
 'PT6_MTP',
 'PT8_MTT',
 'PT7_MTT',
 'PT4_PTP',
 'PT1_CTC',
 'PT1_MTT',
 'PT6_MTT',
 'PT1_PTP',
 'PT4_MTT',
 'PT1_MTP',
 'PT2_PTT']

In [10]:
# PTT (primary tumor tissue)
# PTP (primary para-tumor tissue)
# MTT (metastatic tumor tissue)
# MTP (para-metastatic tumor normal brain tissue)
# CTC (circulating tumor cell)
# 按照我们课题中对组织标本（sample）纳排的标准，我们要求保留的组织标本（sample）能代表体内肿瘤微环境的状态，因此组织类型上我们只保留 PTT (primary tumor tissue) 和 MTT (metastatic tumor tissue)
filtered_column_names = [name for name in unique_column_names if 'PTT' in name or 'MTT' in name]
tissue_name = sorted(filtered_column_names)
tissue_name


['PT1_MTT',
 'PT1_PTT',
 'PT2_PTT',
 'PT3_PTT',
 'PT4_MTT',
 'PT4_PTT',
 'PT6_MTT',
 'PT7_MTT',
 'PT8_MTT',
 'PT9_MTT']

In [11]:
# 获取每个 tissue sample 对应哪些 cell id
tissue_cells = {tissue: [] for tissue in tissue_name}

for cell in count_matrix.columns:
    cell_prefix = cell[:7]
    matched_tissue = next((tissue for tissue in tissue_name if tissue in cell_prefix), None)
    
    if matched_tissue:
        tissue_cells[matched_tissue].append(cell)
        

In [12]:
cell_count_per_tissue = {tissue: len(cells) for tissue, cells in tissue_cells.items()}
cell_count_per_tissue

{'PT1_MTT': 64,
 'PT1_PTT': 40,
 'PT2_PTT': 48,
 'PT3_PTT': 56,
 'PT4_MTT': 64,
 'PT4_PTT': 64,
 'PT6_MTT': 72,
 'PT7_MTT': 32,
 'PT8_MTT': 32,
 'PT9_MTT': 48}

In [13]:
#对每个 tissue sample 构建 AnnData 对象
adata_dict = {}

for tissue, cells in tissue_cells.items():
    # 从原始 count_matrix 中提取相应组织的数据
    tissue_data = count_matrix[cells].T  #行为cell，列为gene
    obs_df = pd.DataFrame(index=cells)
    obs_df['sample_id'] = tissue  # 在obs中包含对应tissue
    obs_df['gse_id'] = "GSE198291" 
    var_df = pd.DataFrame(index=tissue_data.columns)
    adata = ad.AnnData(X=tissue_data.values, obs=obs_df, var=var_df)
    
    # 将当前的 AnnData 对象存储到字典中，并以tissue为键
    adata_dict[tissue] = adata

In [14]:
adata_dict

{'PT1_MTT': AnnData object with n_obs × n_vars = 64 × 22335
     obs: 'sample_id', 'gse_id',
 'PT1_PTT': AnnData object with n_obs × n_vars = 40 × 22335
     obs: 'sample_id', 'gse_id',
 'PT2_PTT': AnnData object with n_obs × n_vars = 48 × 22335
     obs: 'sample_id', 'gse_id',
 'PT3_PTT': AnnData object with n_obs × n_vars = 56 × 22335
     obs: 'sample_id', 'gse_id',
 'PT4_MTT': AnnData object with n_obs × n_vars = 64 × 22335
     obs: 'sample_id', 'gse_id',
 'PT4_PTT': AnnData object with n_obs × n_vars = 64 × 22335
     obs: 'sample_id', 'gse_id',
 'PT6_MTT': AnnData object with n_obs × n_vars = 72 × 22335
     obs: 'sample_id', 'gse_id',
 'PT7_MTT': AnnData object with n_obs × n_vars = 32 × 22335
     obs: 'sample_id', 'gse_id',
 'PT8_MTT': AnnData object with n_obs × n_vars = 32 × 22335
     obs: 'sample_id', 'gse_id',
 'PT9_MTT': AnnData object with n_obs × n_vars = 48 × 22335
     obs: 'sample_id', 'gse_id'}

#**预处理：QC <- doublets <- normalize by length**

In [15]:
# 读取基因长度
gene_lengths = pd.read_csv("/data/wenjiayue/Data/smart-seq/GSE198291/gene.length",index_col="Gene")
gene_lengths

,Length
Gene,
DDX11L1,5425
WASH7P,7802
MIR1302-11,1385
FAM138A,1777
OR4G4P,966
...,...
MT-ND6,525
MT-TE,69
MT-CYB,1141


In [16]:
# 预处理
def preprocess_and_save(adata, filename):
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)

    # MT
    mito_genes = adata.var_names.str.startswith('MT-')
    adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
    adata = adata[adata.obs['percent_mito'] < 0.1]

    # doublets
    scrub = scrublet.Scrublet(adata.X)
    doublet_scores, predicted_doublets = scrub.scrub_doublets()
    adata.obs['scrublet_score'] = doublet_scores
    adata.obs['predicted_doublet'] = predicted_doublets
    threshold = 0.3  # cells with the predicted doubletScore larger than 0.3 were further filtered out
    adata = adata[adata.obs['scrublet_score'] <= threshold]

    # normalize (by length)
    merged_df = adata.var.join(gene_lengths, how='left')
    merged_df['Length'] = merged_df['Length'].fillna(1) #未知基因长度用1填充
    normalized_data = adata.X / merged_df['Length'] .values * np.median(merged_df['Length'] .values) #count除以基因长度在乘以总长度的中位数
    adata.X = normalized_data

    print(f"adata shape: {adata.shape}")
    
    # 保存处理后的对象为 h5ad 文件
    adata.write_h5ad(filename)


In [17]:
# 预处理及保存
save_path = '/data/wenjiayue/Data/smart-seq/GSE198291'
for key, adata in adata_dict.items():
    output_filename = os.path.join(save_path, f"GSE198291_{key}.h5ad")
    preprocess_and_save(adata, output_filename)
    print(f'Anndata for {key} saved as {output_filename}')
    

Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.21
Detected doublet rate = 35.6%
Estimated detectable doublet fraction = 17.8%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 200.0%
Elapsed time: 0.5 seconds
adata shape: (45, 12035)
Anndata for PT1_MTT saved as /data/wenjiayue/Data/smart-seq/GSE198291/GSE198291_PT1_MTT.h5ad
Preprocessing...


/tmp/ipykernel_145153/3519821749.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['scrublet_score'] = doublet_scores
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.15
Detected doublet rate = 72.5%
Estimated detectable doublet fraction = 41.2%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 175.8%
Elapsed time: 0.2 seconds
adata shape: (18, 16704)


/tmp/ipykernel_145153/3519821749.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['scrublet_score'] = doublet_scores
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


Anndata for PT1_PTT saved as /data/wenjiayue/Data/smart-seq/GSE198291/GSE198291_PT1_PTT.h5ad
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.11
Detected doublet rate = 31.2%
Estimated detectable doublet fraction = 21.9%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 142.9%
Elapsed time: 0.2 seconds
adata shape: (47, 13420)
Anndata for PT2_PTT saved as /data/wenjiayue/Data/smart-seq/GSE198291/GSE198291_PT2_PTT.h5ad
Preprocessing...
Simulating doublets...


...

/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/scrublet/helper_functions.py:241: RuntimeWarning: invalid value encountered in log
  gLog = lambda input: np.log(input[1] * np.exp(-input[0]) + input[2])
/tmp/ipykernel_145153/3519821749.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['scrublet_score'] = doublet_scores
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


Calculating doublet scores...
Automatically set threshold at doublet score = 0.17
Detected doublet rate = 96.8%
Estimated detectable doublet fraction = 40.3%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 240.0%
Elapsed time: 0.2 seconds
adata shape: (19, 13173)
Anndata for PT7_MTT saved as /data/wenjiayue/Data/smart-seq/GSE198291/GSE198291_PT7_MTT.h5ad
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.09
Detected doublet rate = 100.0%
Estimated detectable doublet fraction = 93.8%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 106.7%
Elapsed time: 0.1 seconds
adata shape: (23, 18276)


/tmp/ipykernel_145153/3519821749.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['scrublet_score'] = doublet_scores
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


Anndata for PT8_MTT saved as /data/wenjiayue/Data/smart-seq/GSE198291/GSE198291_PT8_MTT.h5ad
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.11
Detected doublet rate = 75.0%
Estimated detectable doublet fraction = 59.4%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 126.3%
Elapsed time: 0.2 seconds
adata shape: (41, 14130)
Anndata for PT9_MTT saved as /data/wenjiayue/Data/smart-seq/GSE198291/GSE198291_PT9_MTT.h5ad


/tmp/ipykernel_145153/3519821749.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['scrublet_score'] = doublet_scores
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/hdd/wenjiayue/Envs/miniconda/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


#**END**